In [2]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
# from torchvision.transforms import ToTensorfrom
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np
import os

from scipy.spatial import KDTree

In [3]:
class KDTreeDataset(Dataset):
    def __init__(self, pts_file, split=0):
        points = np.loadtxt(pts_file, delimiter=' ')
        points = np.delete(points, -2, 1)
        points = np.delete(points, -2, 1)
        # points = np.delete(points, -2, 1)

        if split == 1:
            points = points[:len(points)//2]
        elif split == 2:
            points = points[len(points)//2:]

        print('file points shape', points.shape)
        
        self.dim_scalars = []
        for i in range(4):
            dim_min, dim_max = min(points[:,i]), max(points[:,i])
            points[:,i] = (points[:,i] - dim_min) / (dim_max - dim_min)
            self.dim_scalars.append(dim_max - dim_min)
        self.data = points

        self.tree = KDTree(self.data[:, :3])
        
    def __len__(self):
        return len(self.data) // 15000
        
    def __getitem__(self, idx):
        # Return batches of 25000 points
        # print('Starting point', self.data[idx * 10000, :3])
        noisy_point = self.data[idx * 15000, :3] + np.random.normal(0, 0.01, 3)[0]
        # print('Noisy point', noisy_point)
        
        _, indicies = self.tree.query(noisy_point, k=25000)

        xyzirn = self.data[indicies, :-1]  # x, y, z, ***intensity, return number, number of returns***
        label = self.data[indicies, -1] == 5

        xyzirn = torch.from_numpy(xyzirn.T).float() # intensity, z, y, x from top to bottom? or other way around
        label = torch.tensor(label).long()
        
        # print(xyzirn.size(), label.size())

        return xyzirn, label

class NormalDataset(Dataset):
    def __init__(self, pts_file, split=0):
        points = np.loadtxt(pts_file, delimiter=' ')
        points = np.delete(points, -2, 1)
        points = np.delete(points, -2, 1)
        # points = np.delete(points, -2, 1)

        if split == 1:
            points = points[:len(points)//2]
        elif split == 2:
            points = points[len(points)//2:]

        print(points.shape)
        # do i need to min max intensity, return number, number of returns, etc.? probably not
        self.dim_scalars = []
        for i in range(4):
            dim_min, dim_max = min(points[:,i]), max(points[:,i])
            points[:,i] = (points[:,i] - dim_min) / (dim_max - dim_min)
            self.dim_scalars.append(dim_max - dim_min)
        self.data = points
        
    def __len__(self):
        return len(self.data) // 25000
        
    def __getitem__(self, idx):
        # Return batches of 25000 points
        xyzirn = self.data[idx * 25000: (idx + 1) * 25000, :-1]  # x, y, z, ***intensity, return number, number of returns***
        label = self.data[idx * 25000: (idx + 1) * 25000, -1] == 5

        xyzirn = torch.from_numpy(xyzirn.T).float() # intensity, z, y, x from top to bottom
        label = torch.tensor(label).long()
        
        return xyzirn, label


data_folder = os.path.join(os.getcwd(), r"data\Point Cloud")
training_data = KDTreeDataset(os.path.join(data_folder, r"Traininig.pts"))
validation_data = NormalDataset(os.path.join(data_folder, r"Testing.pts"), split=1)
testing_data = NormalDataset(os.path.join(data_folder, r"Testing.pts"), split=2)
train_dataloader = DataLoader(training_data, batch_size=5, shuffle=True) # can/should i use shuffle, try lowering it?
validation_dataloader = DataLoader(validation_data, batch_size=5, shuffle=False)
test_dataloader = DataLoader(testing_data, batch_size=5, shuffle=False)

num_classes = 2

file points shape (753876, 5)
(205861, 5)
(205861, 5)


In [4]:
print(len(training_data))
print(len(train_dataloader))

50
10


In [5]:
for i, data in enumerate(train_dataloader):
    print(data[0].size())

torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])


In [6]:
import open3d as o3d
import numpy as np
import copy
from collections import Counter

def plot_view(entire_view = True):
    # Assuming pc is your point cloud data, in shape (N, 3)
    pc_num = 2

    if entire_view:
        pc = training_data.data[:, :3]
        labels = training_data.data[:, -1] == 5
    else:
        pc, labels = training_data[pc_num]
        pc = pc.T.numpy()[:, :3]
        labels = labels.numpy()

    pc = copy.deepcopy(pc)
    for i in range(3):
        pc[:, i] *= training_data.dim_scalars[i]

    print(Counter(labels), len(labels))

    # Define colors for each label
    color_map = {0: [0.5, 0.5, 0.5],  # Gray color for label 0
                1: [1.0, 0.0, 0.0]}  # Red color for label 1

    # Map each label to a color
    colors = np.array([color_map[label] for label in labels])

    # Create point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(pc)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])
# plot_view(entire_view = False)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
def plot_batch(data, labels):
    pc, labels = data.transpose(1,2).reshape(-1, 4).numpy()[:, :3], labels.view(-1).numpy()
    # print(labels.shape)
    pc = copy.deepcopy(pc)
    for i in range(3):
        pc[:, i] *= training_data.dim_scalars[i]

    print(Counter(list(labels)), len(labels))

    # Define colors for each label
    color_map = {0: [0.5, 0.5, 0.5],  # Gray color for label 0
                1: [1.0, 0.0, 0.0]}  # Red color for label 1

    # Map each label to a color
    colors = np.array([color_map[label] for label in labels])

    # Create point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(pc)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])
# for i, data in enumerate(train_dataloader):
#     print(data[0].size())
#     plot_batch(*data)
#     break

In [8]:
# Get cpu, gpu or mps device for training.
# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )
device = 'cpu'
print(f"Using {device} device")

Using cpu device


In [9]:
from typing import  NamedTuple

import torch
from torch import Tensor, nn

class _KNN(NamedTuple):
    neighbors: Tensor
    dists: Tensor
    indices: Tensor

def gather_neighbors(x: Tensor, idx: Tensor):
    """
    Gather K nearest neighbors into single tensor.

    Parameters
    ----------
    x : Tensor
        Original data of shape [B, N, C].
    idx : Tensor
        Index positions gathered from topk of the pairwise distance matrix of shape
        [B, N, K].

    Returns
    -------
    Tensor
        K nearest neighbors of shape [B, N, K, C].
    """
    C = x.shape[-1]
    K = idx.shape[-1]
    idx_expanded = idx[:, :, :, None].expand(-1, -1, -1, C)  # B, N, K, C

    return x[:, :, None].expand(-1, -1, K, -1).gather(1, idx_expanded)

def knn(
    x1: Tensor,
    x2: Tensor,
    k: int,
    xyz_only: bool = True,
    self_loop: bool = True,
    topk: bool = False,
) -> _KNN:
    """
    Find the K nearest neighbors in y for every row in x.

    Parameters
    ----------
    x1 : Tensor
        Origin input of shape (*, N, C).
    x2 : Tensor
        Target input of shape (*, M, C).
    k : int
        Number of K neighbors.
    xyz_only : bool
        Whether to use the first 3 channels, defaults to True.
    self_loop : bool
        Whether to include query point in nearest neighbors, defaults to True.
    topk : bool
        If true, calculate Largest K values returned from kernel, else return Smallest
        K, defaults to False.
    **kernel_kwargs : Any
        Keyword arguments to pass to kernel function.

    Returns
    -------
    KNN
        NamedTuple of (neighbors, dists, indices).
    """
    # [B, N, N]
    if xyz_only:
        pdists = torch.cdist(x1[..., :3], x2[..., :3])
    else:
        pdists = torch.cdist(x1, x2)

    if not self_loop:
        fill_value = torch.inf if not topk else -torch.inf
        new_diag = torch.full(tuple(pdists.size()[:2]), fill_value, device=x1.device)
        pdists = pdists.diagonal_scatter(new_diag, dim1=1, dim2=2)

    # [B, N, K]
    dists, idxs = torch.topk(pdists, k=k, largest=topk, dim=-1)

    # [B, N, K, C]
    nearest_neighbors = gather_neighbors(x1, idxs)

    return _KNN(nearest_neighbors, dists, idxs)

class EdgeConv(nn.Module):
    """
    https://arxiv.org/abs/1801.07829.

    Parameters
    ----------
    in_channels: int
        Number of input channels.
    out_channels: int
        Number of output channels.
    k: int
        Number of neighbors to consider, defaults to 20.
    depth: int
        Number of convolutions to apply.
    xyz_only: bool
        If True, only use the first 3 channels for KNN.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        k: int = 20,
        xyz_only: bool = False,
        norm: bool = True,
    ):
        super().__init__()
        self.k = k
        self.xyz_only = xyz_only
        graph_dim = in_channels * 2
        self.mlp = nn.Sequential(
            nn.Linear(graph_dim, out_channels, bias=norm is False),
            nn.LayerNorm(torch.Size((k, out_channels))) if norm else nn.Identity(),
            nn.LeakyReLU(0.2),
            nn.Linear(out_channels, out_channels, bias=norm is False),
            nn.LayerNorm(torch.Size((k, out_channels))) if norm else nn.Identity(),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x: Tensor):
        graph = self._create_graph(x)
        return self.mlp(graph).max(dim=-2)[0]

    def _create_graph(self, x):
        xj = knn(x, x, self.k, self.xyz_only).neighbors
        xi = x.unsqueeze(-2).expand_as(xj)
        return torch.cat((xj - xi, xi), dim=-1)
    
class DGCNNBase(nn.Module):
    def __init__(
        self,
        in_channels: int,
        k: int,
        global_dim: int = 1024,
        edge_conv_dim: int = 64,
        depth: int = 3,
    ):
        super().__init__()
        self.edge_convs = nn.ModuleDict()
        for i in range(depth):
            inc = in_channels if i == 0 else edge_conv_dim
            xyz_only = i == 0
            self.edge_convs[f"graph_{i}"] = EdgeConv(
                in_channels=inc,
                out_channels=edge_conv_dim,
                xyz_only=xyz_only,
                k=k,
            )

        self.global_feature = nn.Sequential(
            nn.Linear(edge_conv_dim * depth, global_dim),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        """
        Forward pass through the model.

        Input: (B, C, N)
        EdgeConv(s): (B, N, C) -> (B, N, edge_conv_dim)
        Global: (B, N, edge_conv_dim * depth) -> (B, 1, global_dim)
        """
        local_features = []
        for edge_conv in self.edge_convs.values():
            x = edge_conv(x)
            local_features.append(x)
        local_features = torch.cat(local_features, dim=-1)
        global_features = self.global_feature(local_features).max(
            dim=-2, keepdims=True
        )[0]
        return local_features, global_features
    
class DGCNNSegmentation(nn.Module):
    """
    https://arxiv.org/abs/1801.07829

    Parameters
    ----------
    in_channels : int
        Number of input channels.
    k : Optional[int]
        K neighbors for EdgeConv, defaults to 20.
    global_dim : Optional[int]
        Features to compute for global features, defaults to 1024.
    edge_conv_dim : Optional[int]
        EdgeConv dimension, defaults to 64.
    depth : Optional[int]
        Number of EdgeConv layers, defaults to 3.
    out_channels : int
        Number of output channels.
    dropout : Optional[float]
        Dropout percentage, defaults to 0.5.
    """

    def __init__(
        self,
        in_channels,
        out_channels,
        k=20,
        global_dim=1024,
        edge_conv_dim=64,
        depth=3,
        dropout=0.0,
    ):
        super().__init__()

        self.base = DGCNNBase(
            in_channels=in_channels,
            k=k,
            global_dim=global_dim,
            edge_conv_dim=edge_conv_dim,
            depth=depth,
        )
        self.head = nn.Sequential(
            nn.Linear(int(global_dim + (edge_conv_dim * depth)), 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout),
            nn.Linear(256, out_channels),
        )

    def forward(self, x):
        print(x.shape)
        local_features, global_features = self.base(x)
        global_features = global_features.repeat(1, x.shape[1], 1)
        x = torch.cat([local_features, global_features], dim=-1)
        out = self.head(x)
        return out

In [10]:
def feature_transform_regularizer(trans):
    d = trans.size(1)
    I = torch.eye(d).unsqueeze(0).to(device)
    loss = torch.linalg.norm(I - torch.bmm(trans, trans.transpose(2,1)), dim=(1,2))
    loss = torch.mean(loss)
    return loss

## Online Code

In [11]:
from sklearn.metrics import f1_score
from collections import Counter


writer = SummaryWriter()
classifier = DGCNNSegmentation(in_channels=4, out_channels=2).to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.0005, betas=(0.9, 0.999))

num_batch = len(training_data)

for epoch in range(120):
    classifier.train()
    train_loss, train_f1, train_acc = 0.0, 0.0, 0.0
    predictions, labels = np.array([]), np.array([])
    for i, data in enumerate(train_dataloader, 1):
        points, target = data
        points, target = points.to(device), target.to(device)
        optimizer.zero_grad()
        pred = classifier(points.permute(0,2,1))
        pred = pred.view(-1, num_classes)
        target = target.view(-1, 1).squeeze()
        print(f'Iteration {i}', pred.size(), target.size())

        loss = loss_func(pred, target)
        loss.backward()
        optimizer.step()
        train_loss += float(loss) # JUST USING LOSS ACCUMULATES HISTORY

        predictions = np.append(predictions, pred.max(1)[1].cpu())
        labels = np.append(labels, target.cpu())

    train_f1 = f1_score(predictions, labels)
    train_acc = sum(predictions == labels)/float(len(labels))
    train_loss /= len(train_dataloader)


    classifier.eval()
    valid_loss, valid_f1, valid_acc = 0.0, 0.0, 0.0
    predictions, labels = np.array([]), np.array([])
    with torch.no_grad():
        for i, data in enumerate(validation_dataloader):
            points, target = data
            points, target = points.to(device), target.to(device)
            pred,  trans_feat = classifier(points.permute(0,2,1))
            pred = pred.view(-1, num_classes)
            target = target.view(-1, 1).squeeze()

            loss = loss_func(pred, target)
            valid_loss += float(loss)

            predictions = np.append(predictions, pred.max(1)[1].cpu())
            labels = np.append(labels, target.cpu())

    valid_f1 = f1_score(predictions, labels)
    valid_acc = sum(predictions == labels)/float(len(labels))
    valid_loss /= len(validation_dataloader)


    writer.add_scalars('losses', {'training':train_loss, 'validation':valid_loss}, global_step=epoch)
    writer.add_scalars('f1 scores', {'training':train_f1, 'validation':valid_f1}, global_step=epoch)

    print(f'[{epoch}] train loss: {train_loss} accuracy: {train_acc} f1 score: {train_f1}')
    print(f'[{epoch}] validation loss: {valid_loss} accuracy: {valid_acc} f1 score: {valid_f1}')
    print()

writer.flush()
writer.close()

torch.Size([5, 25000, 4])
Iteration 1 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 25000, 4])
Iteration 2 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 25000, 4])


In [ ]:
from sklearn.metrics import f1_score

outputs = None
points = None
x = y = None
def test_PointNet(pointnet, test_dataloader, device):
    global outputs, points, x, y
    pointnet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (points, labels) in enumerate(test_dataloader):
            points, labels = points.to(device), labels.to(device)
            outputs, _ = pointnet(points)
            _, predicted = torch.max(outputs.data, 2)
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()
            x = labels.view(-1).to('cpu').numpy()
            y = predicted.view(-1).to('cpu').numpy()
            f1 = f1_score(x, y)
            print('F1 score: ', f1)
            plot_batch(points, predicted)
            plot_batch(points, labels)
    
test_PointNet(classifier, test_dataloader, device)

F1 score:  0.3325457767277023
F1 score:  0.34448641403679786


In [ ]:
print(sum(x))
print(sum(y))
print(outputs)

20070
20965
tensor([[[-0.0199, -3.9263],
         [-0.0199, -3.9261],
         [-0.0199, -3.9259],
         ...,
         [-0.3394, -1.2456],
         [-0.1242, -2.1474],
         [-0.1215, -2.1678]],

        [[-0.1984, -1.7152],
         [-0.1861, -1.7731],
         [-0.1867, -1.7700],
         ...,
         [-0.0231, -3.7794],
         [-0.0232, -3.7755],
         [-0.0233, -3.7693]],

        [[-0.0218, -3.8362],
         [-0.0219, -3.8333],
         [-0.0320, -3.4593],
         ...,
         [-1.0450, -0.4334],
         [-1.0428, -0.4346],
         [-1.0892, -0.4102]]], device='cuda:0')


In [ ]:
print(f"Labels has {sum(x == 0)} zeroes and {sum(x == 1)} ones")
print(f"Predictions has {sum(y == 0)} zeroes and {sum(y == 1)} ones")


# writer.add_graph(model, points)
# writer.flush()
# writer.close()

Labels has 54930 zeroes and 20070 ones
Predictions has 54035 zeroes and 20965 ones


73.4029268292683% on roofs (5)
86.77414634146342% on trees (8)